In [1]:
import jieba
from tqdm import tqdm
import jieba.posseg as pseg

# 预处理

In [2]:
# 根据外部语料和词典 统计用户词典的词频
# 外部语料无格式限定，仅用于统计词频，用户词典一行一词
# 分别命名为 outer_text.txt user_dict.txt
with open('user_dict.txt', 'r') as fr:
    dict_words = fr.readlines()
    dict_words = [word.strip().replace(' ','_') for word in dict_words] # 空格改为下划线, 否则jieba无法识别
    
with open('outer_text.txt', 'r') as fr:
    text = fr.read()


In [3]:
# 统计用户词典在语料中出现的词频
word_shownum = []
for word in tqdm(dict_words):
    word_shownum.append( (word, text.count(word)) )

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30416/30416 [16:27<00:00, 30.81it/s]


In [4]:
# 按照jieba需要的格式写入文件，注意词性可以不填，这里全部以普调名词'n'作为示例
word_shownum.sort(key = lambda x: x[1], reverse = True)
with open('user_dict_tmp.txt', 'w') as fw:
    for item in word_shownum:
        fw.write(item[0] + ' ' + str(item[1]) + ' ' + 'n' + '\n')

In [5]:
word_shownum[:10]

[('淘宝', 31117),
 ('京东', 19630),
 ('淘宝特价', 12609),
 ('得物', 8793),
 ('快手', 8229),
 ('华为', 7387),
 ('老板', 5833),
 ('支付宝', 5705),
 ('极速版', 4885),
 ('饿了么', 4067)]

In [6]:
# 加载字典
jieba.load_userdict('user_dict_tmp.txt')

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/q8/stvv_bwn0hz6fw4p66mwn1lm0000gn/T/jieba.cache
Loading model cost 0.460 seconds.
Prefix dict has been built successfully.


In [7]:
# 分词示例
words = pseg.cut('淘宝京东淘宝特价得物快手老板支付宝')
print(list(words))

[pair('淘宝', 'n'), pair('京东', 'n'), pair('淘宝特价', 'n'), pair('得物', 'n'), pair('快手', 'n'), pair('老板', 'n'), pair('支付宝', 'n')]


In [11]:
# 构建NER函数
entity_set = set(dict_words)

def NER_by_Dict(sent):
    words = pseg.cut(sent)
    for word, flag in words:
        # print(word, flag)
        # print(word in entity_set)
        if flag == 'n' and word in entity_set: # 判断为实体
            
            sent = sent.replace(word, '{实体}', 1)
    return sent  

In [12]:
# 测试
print(NER_by_Dict('淘宝特价来袭 ，还不赶紧来抢购'))

{实体}来袭 ，还不赶紧来抢购
